# 0. 필요모듈 import

In [ ]:
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함
import time
import json

# 1. 접속 가능 여부 파악하기

## 접속하려는 url 파악하기
- 동적크롤링이 필요한 페이지
- 네트워크 검사를 통해 request url, request method, referer, user-agent 파악
- 접근 가능 여부 확인하기 (request code = 200)

In [ ]:
# 카테고리 데이터 가져오기
# categoryId=0부터 categoryId=10까지
url = 'https://api.itemscout.io/api/v2/keyword/trend/?period=DAILY&categoryId=0'

# 헤더정보 필요
header = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'referer': 'https://itemscout.io/'
}

response = requests.get(url, headers=header)
if response.status_code == requests.codes.ok:
    print('접속성공')
response.text

접속성공


'{"status":"success","data":{"trendRankData":[{"keyword":"비타민d","change":"UP","rankedReason":null,"rank":1,"keywordId":474,"searchCount":163000,"productCount":742246,"firstCategory":4936,"competitionIntensity":4.55},{"keyword":"자라","change":"UP","rankedReason":null,"rank":2,"keywordId":1159671,"searchCount":1312600,"productCount":1065276,"firstCategory":687,"competitionIntensity":0.81},{"keyword":"도장","change":"NEW","rankedReason":null,"rank":3,"keywordId":7586,"searchCount":67600,"productCount":1800164,"firstCategory":45698,"competitionIntensity":26.63},{"keyword":"오메가3","change":"NEW","rankedReason":null,"rank":4,"keywordId":740,"searchCount":229400,"productCount":779241,"firstCategory":2441,"competitionIntensity":3.4},{"keyword":"쿨토시","change":"NEW","rankedReason":null,"rank":5,"keywordId":7406,"searchCount":82600,"productCount":780969,"firstCategory":1153,"competitionIntensity":9.45},{"keyword":"비타민c","change":"NO_CHANGE","rankedReason":null,"rank":6,"keywordId":471,"searchCount":1

# 2. Sample data 편하게 보기

## 가독성이 편한 형태로 변환

In [ ]:
# 받아온 결과를 json 형태로 보기 쉽게 변환
data = json.loads(response.text) # json파일을 리스트로 받아옴
data

- 각 제품의 key값  
  {'keyword', 'change', 'rankedReason', 'rank', 'keywordId', 'searchCount', 'productCount', 'firstCategory','competitionIntensity'}  

- 활용할 수 있는 값  
{'keyword', 'change', 'rankedReason', 'searchCount', 'competitionIntensity'}

In [ ]:
# 필요없는 정보 제외

rank_list = data['data']['trendRankData']
rank_list

**변환된 결과의 형태 예시**  

```
[{'keyword': 's24케이스',  
  'change': 'NEW',  
  'rankedReason': None,  
  'rank': 1,  
  'keywordId': 383049886,  
  'searchCount': 26060,  
  'productCount': 327395,  
  'firstCategory': 4423,  
  'competitionIntensity': 12.56},  
 {'keyword': '콘드로이친',  
  'change': 'UP',  
  'rankedReason': None,  
  'rank': 2,  
  'keywordId': 5496680,  
  'searchCount': 166100,  
  'productCount': 206159,  
  'firstCategory': 2601,  
  'competitionIntensity': 1.24},  
  ...]
```



## 추출한 데이터에서 원하는 데이터만 가져오는 반복문

In [ ]:
# 1위부터 10위까지의 키워드만 추출

top10_keyword = []
for i in range(10):
  keyword = rank_list[i]['keyword']
  top10_keyword.append(keyword)

top10_keyword

['포방터닭볶음탕',
 '달력',
 '하이체어',
 's24케이스',
 '한우선물세트',
 '선물세트',
 '스팸선물세트',
 '스피드랙',
 '한우',
 '음식물처리기']

## 여러 가지 데이터를 하나의 데이터프레임으로 가져오기

In [ ]:
# Extract 'keyword' and 'change' values
keyword_change_data = [{'keyword': d['keyword'], 'change': d['change']} for d in rank_list]

# Create a DataFrame
df = pd.DataFrame(keyword_change_data)
df['change'] = df['change'].map({'NEW': '▲', 'UP': '▲', 'DOWN': '▼', 'NO_CHANGE': '-'})

df

,keyword,change
0,어그부츠,▲
1,스타벅스럭키백,▲
2,방한화,▼
3,나이키,▲
4,패딩부츠,▼
5,크록스,▲
6,셀린느가방,▲
7,방한모자,▼
8,까르띠에시계,▼
9,부쉐론,▲


# 3. 일간 트렌드 정보 가져오기

## 페이지에 접속해서 필요한 데이터를 리스트에 저장하기

In [ ]:
# 일간 트렌드 전체 카테고리 가져오기
daily_keyword = []  # keyword만 추출
daily_all = []  # 모든 정보 추출

# 카테고리 수만큼 range값에 삽입
for i in range(11):
  url = f'https://api.itemscout.io/api/v2/keyword/trend/?period=DAILY&categoryId={i}'
  # url 정상적으로 가져오는 것 확인 완료
  # 헤더정보 필요
  header = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'referer': 'https://itemscout.io/'
  }
  response = requests.get(url, headers=header)
  if response.status_code == requests.codes.ok:
      print('접속성공')
  # 받아온 결과를 json 형태로 보기 쉽게 변환
  data = json.loads(response.text) # json파일을 리스트로 받아옴
  rank_list = data['data']['trendRankData']
  # 1위부터 10위까지의 값 추출
  top20_all = []
  top20_keyword = []
  for j in range(20):
    keyword = rank_list[j]['keyword']
    # 단순히 랭킹만 알기 위한 keyword만 수집
    top20_keyword.append(keyword)
    change = rank_list[j]['change']
    # rankedReason = rank_list[j]['rankedReason']
    searchCount = rank_list[j]['searchCount']
    competitionIntensity = rank_list[j]['competitionIntensity']
    # 다양한 정보를 포함하는 dict생성
    trend_data = {'keyword': keyword, 'change': change, 'searchCount' : searchCount, 'competitionIntensity':competitionIntensity}
    top20_all.append(trend_data)

  daily_keyword.append(top20_keyword)
  daily_all.append(top20_all)

접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공


In [ ]:
# # 아래의 명령어로 결과값을 확인할 수 있다.
# daily_keyword, daily_all

## 모든 데이터를 담은 리스트를 데이터프레임으로

In [ ]:
# 각 카테고리에 대한 데이터프레임을 담을 리스트
dfs = []

# 반복문을 통해 데이터프레임 생성 및 리스트에 추가
for i in daily_all:
    df_category = pd.DataFrame(i)
    df_category['change'] = df_category['change'].map({'NEW': '▲', 'UP': '▲', 'DOWN': '▼', 'NO_CHANGE': '-'})
    dfs.append(df_category)

In [ ]:
# 첫 번째 데이터프레임을 기준으로 시작
result_df = dfs[0]

# 나머지 데이터프레임을 반복문을 통해 오른쪽으로 합침
for df in dfs[1:]:
    result_df = pd.concat([result_df, df], axis=1)

result_df

,keyword,change,searchCount,competitionIntensity,keyword,change,searchCount,competitionIntensity,keyword,change,...,searchCount,competitionIntensity,keyword,change,searchCount,competitionIntensity,keyword,change,searchCount,competitionIntensity
0,비타민d,▲,163000,4.55,남자우비,▲,9350,82.24,크록스,▲,...,82600,9.45,도장,▲,67600,26.63,일본esim,▲,32430,0.17
1,자라,▲,1312600,0.81,내셔널지오그래픽,-,180800,3.27,레인부츠,▲,...,132900,13.10,설거지,▼,54950,16.79,문화상품권,▲,137800,0.02
2,도장,▲,67600,26.63,기능성반팔,▲,26570,18.44,루이비통,▲,...,14670,9.98,방충망,▲,109100,11.29,에어컨청소,▼,284700,1.00
3,오메가3,▲,229400,3.40,스투시,▼,474300,0.57,헌터레인부츠,▼,...,203300,35.06,텀블러,▼,179900,18.58,일본유심,▲,75500,0.07
4,쿨토시,▲,82600,9.45,아미반팔티,▲,44430,1.26,장우산,▲,...,274100,6.75,수국,▲,197200,0.95,신세계상품권,▲,82400,0.05
5,비타민c,-,142000,6.67,폴로랄프로렌,▲,259500,3.99,아쿠아슈즈,▲,...,124800,3.54,무인양품,▲,345200,0.31,제주도스냅,▲,3850,3.00
6,크록스,▼,1112400,0.88,스투시반팔,▼,223500,0.19,리복하이페리엄,▲,...,102500,16.23,모기장,▲,111800,19.55,거제소노캄워터파크,▲,14990,0.00
7,레인부츠,▼,157400,9.22,앤아더스토리즈,▲,116700,0.13,런닝화,▲,...,73400,9.97,차량용방향제,▲,140300,5.08,몽골유심,▲,7140,0.05
8,닭발,▲,69510,1.02,판초우의,▼,25520,13.77,여성샌들,▲,...,103100,13.21,빨래건조대,▼,74200,11.68,뉴카멜리아호,▲,19030,0.19
9,창문형에어컨,▼,454400,0.12,여름가디건,▲,94600,11.47,14k반지,▲,...,109100,19.46,멀티탭,▲,108200,15.96,기차여행,▼,40220,0.94


## 일간 트렌드 키워드만 담은 리스트를 데이터프레임으로

In [ ]:
df = pd.DataFrame(data = daily_keyword).T
df.columns=['전체', '패션의류','패션잡화','화장품/미용','디지털/가전','가구/인테리어','출산/육아','식품','스포츠/레저','생활/건강', '여가/편의']
df

,전체,패션의류,패션잡화,화장품/미용,디지털/가전,가구/인테리어,출산/육아,식품,스포츠/레저,생활/건강,여가/편의
0,비타민d,남자우비,크록스,페디큐어,창문형에어컨,집들이선물,아기섬유유연제,오메가3,쿨토시,도장,일본esim
1,자라,내셔널지오그래픽,레인부츠,타투스티커,선풍기,여름이불,아기세제,비타민d,래쉬가드,설거지,문화상품권
2,도장,기능성반팔,루이비통,마데카크림,이동식에어컨,암막커튼,젖병세정제,비타민c,골프스타킹,방충망,에어컨청소
3,오메가3,스투시,헌터레인부츠,율무팩,모기퇴치기,침대프레임,돌답례품,삼계탕,수영복,텀블러,일본유심
4,쿨토시,아미반팔티,장우산,입생로랑,에어컨,식탁의자,아동래쉬가드,복숭아,전기자전거,수국,신세계상품권
5,비타민c,폴로랄프로렌,아쿠아슈즈,샴푸바,무선청소기,아크릴액자,유아구명조끼,쌀국수,원터치텐트,무인양품,제주도스냅
6,크록스,스투시반팔,리복하이페리엄,들깨기미크림,음식물처리기,쿨매트,전동워터건,닭가슴살,비키니,모기장,거제소노캄워터파크
7,레인부츠,앤아더스토리즈,런닝화,홀리카홀리카,보조배터리,컴퓨터의자,미아방지목걸이,떡볶이,구명조끼,차량용방향제,몽골유심
8,닭발,판초우의,여성샌들,네이처리퍼블릭,아이폰케이스,경추베개,층간소음매트,돈까스,아이스박스,빨래건조대,뉴카멜리아호
9,창문형에어컨,여름가디건,14k반지,탈모샴푸,무선선풍기,소파베드,레고,순대국,타프,멀티탭,기차여행


# 결과 저장하기

In [ ]:
# csv 파일로 저장
# 일간트렌드 데이터마다 파일명 다르게 저장
from datetime import datetime
import pytz

# 현재 시간을 가져옵니다.
current_time = datetime.now()

# UTC 시간대로 변경합니다.
utc_time = current_time.replace(tzinfo=pytz.utc)

# 한국 시간대로 변경합니다.
korea_time = utc_time.astimezone(pytz.timezone('Asia/Seoul'))

today_date = korea_time.strftime('%y%m%d')

In [ ]:
writer = pd.ExcelWriter(f'/content/drive/My Drive/trend_tracking/daily_trend_{today_date}.xlsx')
df.to_excel(writer, sheet_name="keyword")
result_df.to_excel(writer, sheet_name="all_datas")
writer.close()

주간트렌드는 url이 '*https://api.itemscout.io/api/v2/keyword/trend/?period=WEEKLY&categoryId=*'형태로 진행됨